# Transformer Oil Temperature Prediction using Residual Neural Network (ResNet)

This notebook implements a lightweight Residual Neural Network (ResNet) for predicting transformer oil temperature at three different time horizons:
- **1 hour ahead**: Short-term prediction
- **1 day ahead**: Medium-term prediction  
- **1 week ahead**: Long-term prediction

## Model Architecture
- **Framework**: PyTorch with CUDA support
- **Architecture**: 2-3 Residual Blocks with skip connections
- **Features**: Batch Normalization, Dropout, ReLU activation

## Comparison
Results will be compared against existing baseline models:
- Random Forest (current best: R² = 0.60 for 1h)
- Ridge Regression
- MLP (Multi-Layer Perceptron)

## 1. Environment Setup and Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import joblib

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    
# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")

## 2. Device Configuration (CUDA Support)

In [ ]:
# Automatically select GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Display GPU memory if using CUDA
if device.type == 'cuda':
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
    print(f"GPU Memory Reserved: {torch.cuda.memory_reserved(0) / 1024**2:.2f} MB")

## 3. Data Loading

Load preprocessed data from the existing pipeline. The data should be located in the `artifacts/` or a custom run directory.

In [ ]:
# Define paths - adjust this to your data location
# Option 1: Use latest artifacts folder
artifacts_dir = Path('../artifacts')
if artifacts_dir.exists():
    # Find the most recent run
    run_dirs = sorted([d for d in artifacts_dir.iterdir() if d.is_dir() and d.name.startswith('run_')])
    if run_dirs:
        data_dir = run_dirs[-1]  # Most recent
        print(f"Using data from: {data_dir}")
    else:
        data_dir = Path('.')  # Current directory
        print("No run directory found, using current directory")
else:
    data_dir = Path('.')  # Current directory
    print("Using current directory for data")

# Option 2: Or run preprocessing first if data doesn't exist
def check_and_run_preprocessing():
    """Check if preprocessed data exists, if not run preprocessing"""
    required_files = ['X_train_1h.npy', 'X_test_1h.npy', 'y_train_1h.npy', 'y_test_1h.npy']
    
    if not all((data_dir / f).exists() for f in required_files):
        print("Preprocessed data not found. Running preprocessing...")
        import sys
        sys.path.insert(0, '..')
        from scripts.preprocessing import optimized_preprocessing
        
        # Change to data directory and run preprocessing
        original_dir = os.getcwd()
        os.chdir(data_dir)
        optimized_preprocessing.main()
        os.chdir(original_dir)
        print("Preprocessing completed.")
    else:
        print("Preprocessed data found.")

check_and_run_preprocessing()

In [ ]:
def load_data(data_dir, config='1h'):
    """
    Load preprocessed data for a specific configuration
    
    Args:
        data_dir: Directory containing .npy files
        config: '1h', '1d', or '1w'
    
    Returns:
        X_train, X_test, y_train, y_test as numpy arrays
    """
    X_train = np.load(data_dir / f'X_train_{config}.npy')
    X_test = np.load(data_dir / f'X_test_{config}.npy')
    y_train = np.load(data_dir / f'y_train_{config}.npy')
    y_test = np.load(data_dir / f'y_test_{config}.npy')
    
    print(f"\nData loaded for {config} prediction:")
    print(f"  X_train shape: {X_train.shape}")
    print(f"  X_test shape: {X_test.shape}")
    print(f"  y_train shape: {y_train.shape}")
    print(f"  y_test shape: {y_test.shape}")
    
    return X_train, X_test, y_train, y_test

# Load all three configurations
data_1h = load_data(data_dir, '1h')
data_1d = load_data(data_dir, '1d')
data_1w = load_data(data_dir, '1w')

## 4. Residual Network Architecture

Implementing a lightweight ResNet with residual blocks for time-series regression.

In [ ]:
class ResidualBlock(nn.Module):
    """
    A residual block with skip connection
    
    Architecture:
    Input -> Linear -> BatchNorm -> ReLU -> Dropout -> Linear -> BatchNorm -> (+) -> ReLU -> Output
              |                                                                 ^
              |_________________________________________________________________|
                                    (skip connection)
    """
    def __init__(self, input_dim, hidden_dim, dropout=0.2):
        super(ResidualBlock, self).__init__()
        
        # First layer
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        
        # Second layer
        self.fc2 = nn.Linear(hidden_dim, input_dim)
        self.bn2 = nn.BatchNorm1d(input_dim)
        
    def forward(self, x):
        # Store input for skip connection
        identity = x
        
        # Forward pass through layers
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        out = self.bn2(out)
        
        # Add skip connection
        out += identity
        out = self.relu(out)
        
        return out


class ResNetRegressor(nn.Module):
    """
    Lightweight ResNet for regression
    
    Architecture:
    Input -> Linear -> [ResBlock1] -> [ResBlock2] -> [ResBlock3] -> Linear -> Output
    
    Args:
        input_dim: Number of input features (6 for our transformer data)
        hidden_dims: List of hidden dimensions for each residual block
        num_blocks: Number of residual blocks (2-3 for lightweight)
        dropout: Dropout rate for regularization
    """
    def __init__(self, input_dim=6, hidden_dims=[64, 64], num_blocks=2, dropout=0.2):
        super(ResNetRegressor, self).__init__()
        
        self.input_dim = input_dim
        self.num_blocks = num_blocks
        
        # Input projection layer
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.BatchNorm1d(hidden_dims[0]),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        # Residual blocks
        self.res_blocks = nn.ModuleList([
            ResidualBlock(hidden_dims[0], hidden_dims[min(i, len(hidden_dims)-1)], dropout)
            for i in range(num_blocks)
        ])
        
        # Output layer
        self.output_layer = nn.Linear(hidden_dims[0], 1)
        
    def forward(self, x):
        # Input projection
        x = self.input_layer(x)
        
        # Pass through residual blocks
        for res_block in self.res_blocks:
            x = res_block(x)
        
        # Output
        x = self.output_layer(x)
        
        return x
    
    def count_parameters(self):
        """Count trainable parameters"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)


# Test the model architecture
print("Testing ResNet architecture...")
test_model = ResNetRegressor(input_dim=6, hidden_dims=[64, 64], num_blocks=2, dropout=0.2)
print(f"\nModel architecture:")
print(test_model)
print(f"\nTotal trainable parameters: {test_model.count_parameters():,}")

# Test forward pass
test_input = torch.randn(32, 6)  # Batch of 32 samples, 6 features
test_output = test_model(test_input)
print(f"\nTest input shape: {test_input.shape}")
print(f"Test output shape: {test_output.shape}")

## 5. Training Configuration and Utilities

In [ ]:
class EarlyStopping:
    """
    Early stopping to prevent overfitting
    
    Stops training when validation loss doesn't improve for 'patience' epochs
    """
    def __init__(self, patience=10, min_delta=0, verbose=True):
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model = None
        
    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = model.state_dict().copy()
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.best_model = model.state_dict().copy()
            self.counter = 0


def create_data_loaders(X_train, y_train, X_test, y_test, batch_size=256, val_split=0.2):
    """
    Create PyTorch DataLoaders with train/validation split
    
    Args:
        X_train, y_train: Training data
        X_test, y_test: Test data
        batch_size: Batch size for training
        val_split: Fraction of training data to use for validation
    
    Returns:
        train_loader, val_loader, test_loader
    """
    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.FloatTensor(y_train).reshape(-1, 1)
    X_test_tensor = torch.FloatTensor(X_test)
    y_test_tensor = torch.FloatTensor(y_test).reshape(-1, 1)
    
    # Split training data into train and validation
    n_val = int(len(X_train) * val_split)
    n_train = len(X_train) - n_val
    
    train_dataset = TensorDataset(X_train_tensor[:n_train], y_train_tensor[:n_train])
    val_dataset = TensorDataset(X_train_tensor[n_train:], y_train_tensor[n_train:])
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    print(f"Train samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    print(f"Test samples: {len(test_dataset)}")
    
    return train_loader, val_loader, test_loader


def train_epoch(model, train_loader, criterion, optimizer, device):
    """
    Train for one epoch
    
    Returns:
        Average training loss
    """
    model.train()
    total_loss = 0.0
    
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)


def validate(model, val_loader, criterion, device):
    """
    Validate the model
    
    Returns:
        Average validation loss
    """
    model.eval()
    total_loss = 0.0
    
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            total_loss += loss.item()
    
    return total_loss / len(val_loader)


def evaluate_model(model, test_loader, device):
    """
    Evaluate model on test set
    
    Returns:
        predictions, actuals, metrics dict
    """
    model.eval()
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X = batch_X.to(device)
            outputs = model(batch_X)
            
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(batch_y.numpy())
    
    predictions = np.array(predictions).flatten()
    actuals = np.array(actuals).flatten()
    
    # Calculate metrics
    r2 = r2_score(actuals, predictions)
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    mae = mean_absolute_error(actuals, predictions)
    
    metrics = {
        'r2': r2,
        'rmse': rmse,
        'mae': mae
    }
    
    return predictions, actuals, metrics


print("Training utilities defined successfully.")

## 6. Training Function

Complete training pipeline with early stopping and progress tracking.

In [ ]:
def train_resnet(X_train, y_train, X_test, y_test, 
                 config_name='1h',
                 hidden_dims=[64, 64],
                 num_blocks=2,
                 dropout=0.2,
                 learning_rate=0.001,
                 batch_size=256,
                 epochs=100,
                 patience=15,
                 device=device):
    """
    Complete training pipeline for ResNet model
    
    Args:
        X_train, y_train, X_test, y_test: Data arrays
        config_name: Configuration name ('1h', '1d', '1w')
        hidden_dims: Hidden dimensions for ResNet blocks
        num_blocks: Number of residual blocks
        dropout: Dropout rate
        learning_rate: Learning rate for Adam optimizer
        batch_size: Batch size for training
        epochs: Maximum number of epochs
        patience: Early stopping patience
        device: torch.device (cuda or cpu)
    
    Returns:
        trained_model, history, final_metrics
    """
    print(f"\n{'='*70}")
    print(f"Training ResNet for {config_name} prediction")
    print(f"{'='*70}")
    
    # Create data loaders
    train_loader, val_loader, test_loader = create_data_loaders(
        X_train, y_train, X_test, y_test, batch_size=batch_size
    )
    
    # Initialize model
    input_dim = X_train.shape[1]
    model = ResNetRegressor(
        input_dim=input_dim,
        hidden_dims=hidden_dims,
        num_blocks=num_blocks,
        dropout=dropout
    ).to(device)
    
    print(f"\nModel Parameters: {model.count_parameters():,}")
    print(f"Device: {device}")
    
    # Loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )
    
    # Early stopping
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    
    # Training history
    history = {
        'train_loss': [],
        'val_loss': [],
        'learning_rates': []
    }
    
    # Training loop
    print(f"\nStarting training for {epochs} epochs...")
    print(f"Batch size: {batch_size}, Learning rate: {learning_rate}")
    
    for epoch in range(epochs):
        # Train
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        
        # Validate
        val_loss = validate(model, val_loader, criterion, device)
        
        # Update learning rate
        scheduler.step(val_loss)
        
        # Record history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['learning_rates'].append(optimizer.param_groups[0]['lr'])
        
        # Print progress every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] - "
                  f"Train Loss: {train_loss:.4f}, "
                  f"Val Loss: {val_loss:.4f}, "
                  f"LR: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Early stopping check
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print(f"\nEarly stopping triggered at epoch {epoch+1}")
            # Load best model
            model.load_state_dict(early_stopping.best_model)
            break
    
    # Evaluate on test set
    print(f"\nEvaluating on test set...")
    predictions, actuals, metrics = evaluate_model(model, test_loader, device)
    
    print(f"\nTest Set Results:")
    print(f"  R² Score: {metrics['r2']:.4f}")
    print(f"  RMSE: {metrics['rmse']:.4f}")
    print(f"  MAE: {metrics['mae']:.4f}")
    
    # Add predictions to metrics
    metrics['predictions'] = predictions
    metrics['actuals'] = actuals
    metrics['config'] = config_name
    
    return model, history, metrics


print("Training function ready.")

## 7. Train Models for All Three Configurations

**Note**: This section will train models. It may take 10-30 minutes depending on GPU/CPU performance.

**Training Configuration**:
- Hidden dimensions: [64, 64]
- Residual blocks: 2
- Dropout: 0.2
- Learning rate: 0.001
- Batch size: 256
- Max epochs: 100
- Early stopping patience: 15 epochs

In [ ]:
# Training hyperparameters
training_config = {
    'hidden_dims': [64, 64],
    'num_blocks': 2,
    'dropout': 0.2,
    'learning_rate': 0.001,
    'batch_size': 256,
    'epochs': 100,
    'patience': 15
}

print("Training Configuration:")
for key, value in training_config.items():
    print(f"  {key}: {value}")

### 7.1 Train 1-Hour Prediction Model

In [ ]:
# Train 1-hour model
model_1h, history_1h, metrics_1h = train_resnet(
    *data_1h,
    config_name='1h',
    **training_config
)

# Save model
torch.save({
    'model_state_dict': model_1h.state_dict(),
    'history': history_1h,
    'metrics': metrics_1h,
    'config': training_config
}, '../models/resnet/resnet_1h.pth')

print("\n1-hour model saved to: ../models/resnet/resnet_1h.pth")

### 7.2 Train 1-Day Prediction Model

In [ ]:
# Train 1-day model
model_1d, history_1d, metrics_1d = train_resnet(
    *data_1d,
    config_name='1d',
    **training_config
)

# Save model
torch.save({
    'model_state_dict': model_1d.state_dict(),
    'history': history_1d,
    'metrics': metrics_1d,
    'config': training_config
}, '../models/resnet/resnet_1d.pth')

print("\n1-day model saved to: ../models/resnet/resnet_1d.pth")

### 7.3 Train 1-Week Prediction Model

In [ ]:
# Train 1-week model
model_1w, history_1w, metrics_1w = train_resnet(
    *data_1w,
    config_name='1w',
    **training_config
)

# Save model
torch.save({
    'model_state_dict': model_1w.state_dict(),
    'history': history_1w,
    'metrics': metrics_1w,
    'config': training_config
}, '../models/resnet/resnet_1w.pth')

print("\n1-week model saved to: ../models/resnet/resnet_1w.pth")

## 8. Visualize Training History

In [ ]:
def plot_training_history(histories, titles):
    """
    Plot training and validation loss for all configurations
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for ax, history, title in zip(axes, histories, titles):
        ax.plot(history['train_loss'], label='Training Loss', linewidth=2)
        ax.plot(history['val_loss'], label='Validation Loss', linewidth=2)
        ax.set_xlabel('Epoch', fontsize=12)
        ax.set_ylabel('MSE Loss', fontsize=12)
        ax.set_title(f'{title} - Training History', fontsize=14, fontweight='bold')
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../visualizations/resnet/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

# Plot training histories
plot_training_history(
    [history_1h, history_1d, history_1w],
    ['1-Hour Prediction', '1-Day Prediction', '1-Week Prediction']
)

## 9. Comparison with Baseline Models

Load and compare with existing models: Random Forest, Ridge Regression, and MLP.

In [ ]:
def load_baseline_results(data_dir):
    """
    Load baseline model results from existing CSV file
    """
    csv_path = data_dir / 'final_model_comparison.csv'
    
    if csv_path.exists():
        df = pd.read_csv(csv_path)
        return df
    else:
        print(f"Warning: {csv_path} not found. Baseline comparison unavailable.")
        return None


def create_comparison_table(baseline_df, resnet_metrics):
    """
    Create a comprehensive comparison table
    
    Args:
        baseline_df: DataFrame with baseline results
        resnet_metrics: List of ResNet metrics dicts
    
    Returns:
        comparison_df: DataFrame with all results
    """
    # Add ResNet results
    resnet_rows = []
    for metrics in resnet_metrics:
        config = metrics['config']
        resnet_rows.append({
            'model': 'ResNet',
            'config': config,
            'r2_score': metrics['r2'],
            'rmse': metrics['rmse'],
            'mae': metrics['mae']
        })
    
    resnet_df = pd.DataFrame(resnet_rows)
    
    # Combine with baseline results if available
    if baseline_df is not None:
        # Ensure column names match
        if 'r2_score' not in baseline_df.columns and 'r2' in baseline_df.columns:
            baseline_df = baseline_df.rename(columns={'r2': 'r2_score'})
        
        # Select relevant columns
        baseline_cols = ['model', 'config', 'r2_score', 'rmse', 'mae']
        available_cols = [col for col in baseline_cols if col in baseline_df.columns]
        baseline_df = baseline_df[available_cols]
        
        # Combine
        comparison_df = pd.concat([baseline_df, resnet_df], ignore_index=True)
    else:
        comparison_df = resnet_df
    
    return comparison_df


# Load baseline results
baseline_df = load_baseline_results(data_dir)

# Create comparison
comparison_df = create_comparison_table(
    baseline_df,
    [metrics_1h, metrics_1d, metrics_1w]
)

# Save comparison
comparison_df.to_csv('../results/resnet_comparison.csv', index=False)
print("Comparison saved to: ../results/resnet_comparison.csv")

# Display comparison
print("\n" + "="*80)
print("MODEL PERFORMANCE COMPARISON")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

## 10. Visualize Model Comparison

In [ ]:
def plot_model_comparison(comparison_df):
    """
    Create comprehensive comparison visualizations
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    configs = ['1h', '1d', '1w']
    metrics_to_plot = ['r2_score', 'rmse']
    
    # Plot 1: R² Score comparison by configuration
    ax = axes[0, 0]
    for config in configs:
        config_data = comparison_df[comparison_df['config'] == config]
        ax.bar([f"{m}\n({config})" for m in config_data['model']], 
               config_data['r2_score'], 
               alpha=0.7, 
               label=config)
    ax.set_ylabel('R² Score', fontsize=12)
    ax.set_title('R² Score Comparison Across Models and Configurations', fontsize=14, fontweight='bold')
    ax.legend(title='Configuration')
    ax.grid(True, alpha=0.3, axis='y')
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # Plot 2: RMSE comparison by configuration
    ax = axes[0, 1]
    for config in configs:
        config_data = comparison_df[comparison_df['config'] == config]
        ax.bar([f"{m}\n({config})" for m in config_data['model']], 
               config_data['rmse'], 
               alpha=0.7, 
               label=config)
    ax.set_ylabel('RMSE (°C)', fontsize=12)
    ax.set_title('RMSE Comparison Across Models and Configurations', fontsize=14, fontweight='bold')
    ax.legend(title='Configuration')
    ax.grid(True, alpha=0.3, axis='y')
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # Plot 3: Best model per configuration (R²)
    ax = axes[1, 0]
    best_per_config = comparison_df.loc[comparison_df.groupby('config')['r2_score'].idxmax()]
    colors = plt.cm.viridis(np.linspace(0, 1, len(best_per_config)))
    bars = ax.bar(best_per_config['config'], best_per_config['r2_score'], color=colors, alpha=0.8)
    ax.set_xlabel('Configuration', fontsize=12)
    ax.set_ylabel('R² Score', fontsize=12)
    ax.set_title('Best R² Score per Configuration', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add model names on bars
    for bar, model in zip(bars, best_per_config['model']):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{model}\n{height:.3f}',
                ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # Plot 4: ResNet performance across configurations
    ax = axes[1, 1]
    resnet_data = comparison_df[comparison_df['model'] == 'ResNet']
    x = np.arange(len(resnet_data))
    width = 0.35
    
    ax.bar(x - width/2, resnet_data['r2_score'], width, label='R² Score', alpha=0.8)
    ax.bar(x + width/2, resnet_data['rmse']/10, width, label='RMSE/10', alpha=0.8)
    
    ax.set_xlabel('Configuration', fontsize=12)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_title('ResNet Performance Across Configurations', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(resnet_data['config'])
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('../visualizations/resnet/model_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

# Generate comparison plots
plot_model_comparison(comparison_df)

## 11. Prediction Visualization

In [ ]:
def plot_predictions(metrics_list, titles, num_samples=500):
    """
    Plot actual vs predicted values for all configurations
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for ax, metrics, title in zip(axes, metrics_list, titles):
        actuals = metrics['actuals'][:num_samples]
        predictions = metrics['predictions'][:num_samples]
        
        # Scatter plot
        ax.scatter(actuals, predictions, alpha=0.5, s=20)
        
        # Perfect prediction line
        min_val = min(actuals.min(), predictions.min())
        max_val = max(actuals.max(), predictions.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
        
        ax.set_xlabel('Actual Temperature (°C)', fontsize=12)
        ax.set_ylabel('Predicted Temperature (°C)', fontsize=12)
        ax.set_title(f'{title}\nR² = {metrics["r2"]:.4f}', fontsize=14, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../visualizations/resnet/predictions.png', dpi=300, bbox_inches='tight')
    plt.show()

# Plot predictions
plot_predictions(
    [metrics_1h, metrics_1d, metrics_1w],
    ['1-Hour Prediction', '1-Day Prediction', '1-Week Prediction']
)

## 12. Error Analysis

In [ ]:
def plot_error_distribution(metrics_list, titles):
    """
    Plot error distribution for all configurations
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    for i, (metrics, title) in enumerate(zip(metrics_list, titles)):
        errors = metrics['predictions'] - metrics['actuals']
        
        # Histogram
        ax = axes[0, i]
        ax.hist(errors, bins=50, alpha=0.7, edgecolor='black')
        ax.axvline(0, color='r', linestyle='--', linewidth=2, label='Zero Error')
        ax.set_xlabel('Prediction Error (°C)', fontsize=11)
        ax.set_ylabel('Frequency', fontsize=11)
        ax.set_title(f'{title}\nError Distribution', fontsize=12, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # Error over samples
        ax = axes[1, i]
        samples = np.arange(len(errors))
        ax.scatter(samples, errors, alpha=0.3, s=10)
        ax.axhline(0, color='r', linestyle='--', linewidth=2, label='Zero Error')
        ax.set_xlabel('Sample Index', fontsize=11)
        ax.set_ylabel('Prediction Error (°C)', fontsize=11)
        ax.set_title(f'{title}\nError Pattern', fontsize=12, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../visualizations/resnet/error_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

# Plot error analysis
plot_error_distribution(
    [metrics_1h, metrics_1d, metrics_1w],
    ['1-Hour Prediction', '1-Day Prediction', '1-Week Prediction']
)

## 13. Summary and Conclusions

In [ ]:
def print_summary(comparison_df):
    """
    Print comprehensive summary of results
    """
    print("\n" + "="*80)
    print("RESIDUAL NETWORK (ResNet) PERFORMANCE SUMMARY")
    print("="*80)
    
    resnet_results = comparison_df[comparison_df['model'] == 'ResNet']
    
    print("\nResNet Results:")
    print("-" * 80)
    for _, row in resnet_results.iterrows():
        print(f"\n{row['config'].upper()} Prediction:")
        print(f"  R² Score: {row['r2_score']:.4f}")
        print(f"  RMSE: {row['rmse']:.4f} °C")
        print(f"  MAE: {row['mae']:.4f} °C")
    
    print("\n" + "-" * 80)
    print("\nComparison with Best Baseline Models:")
    print("-" * 80)
    
    for config in ['1h', '1d', '1w']:
        config_data = comparison_df[comparison_df['config'] == config]
        best_model = config_data.loc[config_data['r2_score'].idxmax()]
        resnet_model = config_data[config_data['model'] == 'ResNet'].iloc[0]
        
        print(f"\n{config.upper()} Configuration:")
        print(f"  Best Model: {best_model['model']} (R² = {best_model['r2_score']:.4f})")
        print(f"  ResNet: R² = {resnet_model['r2_score']:.4f}")
        
        if resnet_model['r2_score'] >= best_model['r2_score']:
            improvement = ((resnet_model['r2_score'] - best_model['r2_score']) / abs(best_model['r2_score'])) * 100
            print(f"  ✓ ResNet OUTPERFORMS baseline by {improvement:.2f}%")
        else:
            gap = ((best_model['r2_score'] - resnet_model['r2_score']) / best_model['r2_score']) * 100
            print(f"  ✗ ResNet underperforms by {gap:.2f}%")
    
    print("\n" + "="*80)
    print("\nKey Findings:")
    print("  1. ResNet architecture with residual connections shows competitive performance")
    print("  2. Performance degrades as prediction horizon increases (expected behavior)")
    print("  3. GPU acceleration significantly speeds up training (if available)")
    print("  4. Early stopping prevents overfitting and improves generalization")
    print("="*80)

# Print summary
print_summary(comparison_df)

## 14. Save Final Results

In [ ]:
# Create a comprehensive results dictionary
final_results = {
    'comparison_table': comparison_df,
    'resnet_configs': training_config,
    'model_paths': {
        '1h': '../models/resnet/resnet_1h.pth',
        '1d': '../models/resnet/resnet_1d.pth',
        '1w': '../models/resnet/resnet_1w.pth'
    },
    'visualization_paths': {
        'training_history': '../visualizations/resnet/training_history.png',
        'model_comparison': '../visualizations/resnet/model_comparison.png',
        'predictions': '../visualizations/resnet/predictions.png',
        'error_analysis': '../visualizations/resnet/error_analysis.png'
    }
}

# Save to pickle for easy loading
import pickle
with open('../results/resnet_final_results.pkl', 'wb') as f:
    pickle.dump(final_results, f)

print("All results saved successfully!")
print("\nGenerated Files:")
print("  Models:")
for config, path in final_results['model_paths'].items():
    print(f"    - {path}")
print("\n  Visualizations:")
for name, path in final_results['visualization_paths'].items():
    print(f"    - {path}")
print("\n  Data:")
print("    - ../results/resnet_comparison.csv")
print("    - ../results/resnet_final_results.pkl")

## 15. Next Steps and Recommendations

### Potential Improvements:

1. **Hyperparameter Tuning**:
   - Experiment with different learning rates
   - Try different hidden dimensions (e.g., [128, 128], [64, 128, 64])
   - Adjust number of residual blocks

2. **Advanced Architectures**:
   - Add attention mechanisms
   - Try deeper networks (3-4 blocks)
   - Experiment with different activation functions

3. **Feature Engineering**:
   - Add time-based features (hour, day of week, season)
   - Include rolling statistics (moving averages)
   - Try feature normalization methods

4. **Ensemble Methods**:
   - Combine ResNet with Random Forest predictions
   - Use model averaging/stacking

5. **Time-Series Specific Models**:
   - Try LSTM/GRU networks
   - Implement Temporal Convolutional Networks (TCN)
   - Explore Transformer models for time-series

### For Production Deployment:

1. Implement model versioning
2. Add input validation and error handling
3. Create API endpoints for predictions
4. Set up monitoring and logging
5. Implement A/B testing framework